In [1]:
import numpy as np
import pymanopt

In [2]:

def generate_flag_data_noisy(n_pts: int, n: int, flag_type: list, noise: float, seed: int = 1) -> list:
    np.random.seed(seed)

    k = flag_type[-1]
    center_pt = np.linalg.qr(np.random.rand(n, k)-.5)[0][:,:k]

    data = []
    for i in range(n_pts):
        rand_mat = center_pt + noise*(np.random.rand(n, k)-.5)
        data.append(np.linalg.qr(rand_mat)[0][:,:k])

    return data, center_pt

In [3]:
flag_type = [1,4,6]
n = 10
n_pts = 100


data, center_pt = generate_flag_data_noisy(n_pts, n, flag_type, 1e-1)
data = np.stack(data, axis = 2)



In [4]:
verbosity = 1

In [5]:


n,k,p = data.shape

#construct weight matrix
weight_mat = np.eye(p)

    
p_mats = []
id_mats = []

for i in range(len(flag_type)):

    #set the initial f_type_prev to 0
    f_type = flag_type[i]
    if i-1 < 0:
        f_type_prev = 0
    else:
        f_type_prev = flag_type[i-1]
        
    #make projection matrices
    dim_d_mat = data[:,f_type_prev:f_type,:] @ weight_mat
    dim_d_mat = np.reshape(dim_d_mat, (n,(f_type-f_type_prev)*p))
    p_mat = dim_d_mat @ dim_d_mat.T 
    p_mats.append(p_mat)







In [6]:
flag_type0 = [0]+flag_type
m_is = [flag_type0[i]-flag_type0[i-1] for i in range(1,len(flag_type0))]
man_list = [pymanopt.manifolds.Grassmann(n,m_i) for m_i in m_is]
product_grassmann = pymanopt.manifolds.Product(man_list)

In [13]:
product_grassmann

In [19]:
#setup up the objective function
@pymanopt.function.autograd(product_grassmann)
def cost(point):
    f = 0
    for i in range(len(flag_type)):
        f -= np.trace(point[i].T @ p_mats[i] @ point[i])

    return f

In [20]:
problem = pymanopt.Problem(product_grassmann, cost)

In [21]:
# optimizer = pymanopt.optimizers.trust_regions.TrustRegions(verbosity = verbosity)
optimizer = pymanopt.optimizers.conjugate_gradient.ConjugateGradient(verbosity = verbosity)
# optimizer = pymanopt.optimizers.conjugate_gradient.ConjugateGradient(verbosity = verbosity, max_iterations = 20)

#run the trust regions algorithm
# mu = np.mean(data, axis = 2)
# initial_point_all = np.linalg.qr(mu)[0][:,:flag_type[-1]]
# initial_point = []
# for i in range(len(flag_type)):
#     initial_point.append(initial_point_all[:,flag_type0[i]:flag_type0[i+1]])
initial_point = product_grassmann.random_point()
    
result = optimizer.run(problem, initial_point = initial_point)


f_mean = result.point

Optimizing...


TypeError: cost() takes 1 positional argument but 3 were given

In [23]:
initial_point

[array([[ 0.23195153],
        [ 0.46199246],
        [-0.14549372],
        [-0.47215711],
        [ 0.21898411],
        [ 0.55661615],
        [-0.0291176 ],
        [ 0.24187413],
        [-0.25744391],
        [ 0.07250389]]),
 array([[-0.11824906, -0.11451024, -0.4382444 ],
        [-0.10857468, -0.13106461, -0.02834155],
        [ 0.03603835,  0.32139496,  0.24803607],
        [-0.25283041, -0.09982131, -0.68712267],
        [ 0.26605004,  0.38091062, -0.33464207],
        [-0.3479827 ,  0.35692242,  0.08690005],
        [ 0.02198959, -0.70071436,  0.0214802 ],
        [ 0.48166339,  0.14764661, -0.36679246],
        [ 0.43936229,  0.08782596, -0.03256758],
        [ 0.54000903, -0.25189366,  0.13372906]]),
 array([[-0.31150837,  0.32165818],
        [-0.13964477, -0.26488342],
        [ 0.52255234, -0.17095333],
        [-0.24469844, -0.30573346],
        [-0.19501027,  0.37415411],
        [-0.02409543, -0.2808404 ],
        [ 0.4032629 , -0.41976713],
        [-0.01861728,  0

In [22]:
cost

In [10]:
initial_point

[array([[-0.09899253],
        [-0.3580259 ],
        [-0.34170832],
        [-0.41092965],
        [ 0.43468024],
        [-0.4687383 ],
        [ 0.21289779],
        [-0.25258335],
        [-0.24199867],
        [-0.00636455]]),
 array([[ 0.27072652, -0.32210475,  0.29926746],
        [-0.03647827, -0.17677004,  0.14414903],
        [ 0.51404727, -0.16781524, -0.05327291],
        [-0.17143229,  0.11150912, -0.38210979],
        [ 0.27065309, -0.15556657,  0.35341832],
        [ 0.07617827,  0.41720497,  0.27249984],
        [ 0.2797354 , -0.24715553, -0.50159886],
        [ 0.39758461, -0.1552712 ,  0.20017337],
        [-0.30944944, -0.69573851, -0.16721626],
        [-0.46968315, -0.23853012,  0.47119147]]),
 array([[-0.32133509,  0.41668337],
        [-0.0007772 , -0.43899074],
        [-0.16097555, -0.27883428],
        [-0.30136809, -0.35442798],
        [-0.33004496, -0.46756209],
        [ 0.15101354,  0.05855173],
        [ 0.49390558, -0.20070137],
        [ 0.50107503,  0

In [13]:
initial_point

[array([[-0.09899253],
        [-0.3580259 ],
        [-0.34170832],
        [-0.41092965],
        [ 0.43468024],
        [-0.4687383 ],
        [ 0.21289779],
        [-0.25258335],
        [-0.24199867],
        [-0.00636455]]),
 array([[ 0.27072652, -0.32210475,  0.29926746],
        [-0.03647827, -0.17677004,  0.14414903],
        [ 0.51404727, -0.16781524, -0.05327291],
        [-0.17143229,  0.11150912, -0.38210979],
        [ 0.27065309, -0.15556657,  0.35341832],
        [ 0.07617827,  0.41720497,  0.27249984],
        [ 0.2797354 , -0.24715553, -0.50159886],
        [ 0.39758461, -0.1552712 ,  0.20017337],
        [-0.30944944, -0.69573851, -0.16721626],
        [-0.46968315, -0.23853012,  0.47119147]]),
 array([[-0.32133509,  0.41668337],
        [-0.0007772 , -0.43899074],
        [-0.16097555, -0.27883428],
        [-0.30136809, -0.35442798],
        [-0.33004496, -0.46756209],
        [ 0.15101354,  0.05855173],
        [ 0.49390558, -0.20070137],
        [ 0.50107503,  0

In [ ]:

    n,k,p = data.shape

    #construct weight matrix
    weight_mat = np.eye(p)
    if len(weights) > 0:
         weight_mat[np.arange(p), np.arange(p)] = np.sqrt(weights)

    
    p_mats = []
    id_mats = []

    for i in range(len(flag_type)):

        #set the initial f_type_prev to 0
        f_type = flag_type[i]
        if i-1 < 0:
            f_type_prev = 0
        else:
            f_type_prev = flag_type[i-1]
        
        #make projection matrices
        dim_d_mat = data[:,f_type_prev:f_type,:] @ weight_mat
        dim_d_mat = np.reshape(dim_d_mat, (n,(f_type-f_type_prev)*p))
        p_mat = dim_d_mat @ dim_d_mat.T 
        p_mats.append(p_mat)

        #make identity matrices
        id_mat = np.zeros((k,k))
        id_mat[np.arange(f_type_prev,f_type,1),np.arange(f_type_prev,f_type,1)] = 1
        id_mats.append(id_mat)

    if manifold == 'stiefel':
        # initialize a stiefel manifold object
        St = pymanopt.manifolds.stiefel.Stiefel(n,k)
    elif manifold == 'flag':
        # get proper flag type
        
        real_flag_type = []
        real_flag_type.append(flag_type[0])
        for i in range(1,len(flag_type)):
            real_flag_type.append(flag_type[i] - flag_type[i-1])
        real_flag_type.append(n - flag_type[-1])
        real_flag_type.reverse()

        print(real_flag_type)        

        # initialize a flag manifold object
        St = RealFlag(np.array(real_flag_type))

    #setu up the objective function
    @pymanopt.function.autograd(St)
    def cost(point):
        f = 0
        for i in np.arange(len(p_mats)):
            if i < 1:
                f_type_before = 0
            else:
                f_type_before = flag_type[i-1]

            k_i = flag_type[i] - f_type_before
            
            f += p*k_i-np.trace(id_mats[i] @ point.T @ p_mats[i] @ point)

        return f

    problem = pymanopt.Problem(St, cost)

# , max_iterations = 20, max_time = 20)
    optimizer = pymanopt.optimizers.trust_regions.TrustRegions(verbosity = verbosity)
    #optimizer = pymanopt.optimizers.conjugate_gradient.ConjugateGradient(verbosity = verbosity)
    # optimizer = pymanopt.optimizers.conjugate_gradient.ConjugateGradient(verbosity = verbosity, max_iterations = 20)

    #run the trust regions algorithm
    if initial_point is None:
        mu = np.mean(data, axis = 2)
        initial_point = np.linalg.qr(mu)[0][:,:flag_type[-1]]
        result = optimizer.run(problem, initial_point = initial_point)
    else:
        result = optimizer.run(problem, initial_point = initial_point)
    
    f_mean = result.point



In [ ]:
import sys
sys.path.append('../scripts')

import fl_algorithms as fla
import center_algorithms as ca

from matplotlib import pyplot as plt

import numpy as np

import pandas as pd

def generate_flag_data_noisy(n_pts: int, n: int, flag_type: list, noise: float, seed: int = 1) -> list:
    np.random.seed(seed)

    k = flag_type[-1]
    center_pt = np.linalg.qr(np.random.rand(n, k)-.5)[0][:,:k]

    data = []
    for i in range(n_pts):
        rand_mat = center_pt + noise*(np.random.rand(n, k)-.5)
        data.append(np.linalg.qr(rand_mat)[0][:,:k])

    return data, center_pt
        

def generate_flag_data_outliers(n_inliers: int, n_outliers: int, flag_type: list, seed: int = 2) -> list:
    np.random.seed(seed)

    k = flag_type[-1]
    center_pt = np.linalg.qr(np.random.rand(n, k)-.5)[0][:,:k]

    data = []
    for i in range(n_inliers):
        rand_mat = center_pt + 0.001*(np.random.rand(n, k)-.5)
        data.append(np.linalg.qr(rand_mat)[0][:,:k])
    for i in range(n_outliers):
        rand_mat = center_pt + (np.random.rand(n, k)-.5)
        data.append(np.linalg.qr(rand_mat)[0][:,:k])

    return data, center_pt



if __name__ == '__main__':
    
    n_pts = 100
    n = 10
    flag_type = [1,3]   

    n_its = 10

    k = flag_type[-1]
 
    m_errs = []
    med_errs = []
    rfm_errs = []
    rfmed_errs = []
    n_errs = []
    e_errs = []

    noises = []

    #for exp in range(1,100,5):
        #noise = exp/50
        #noises.append(noise)

        #data, center_pt = generate_flag_data(n_pts, n, flag_type, noise)
    for n_outliers in range(20):
        noises.append(n_outliers/n_pts)
        data, center_pt = generate_flag_data_outliers(n_pts-n_outliers, n_outliers, flag_type)
        stacked_data = np.stack(data, axis = 2)

        real_flag_mean = fla.flag_mean(stacked_data,  flag_type = flag_type)  

        real_flag_median = fla.flag_median(stacked_data,  flag_type = flag_type, max_iters = 100)

        #distances to center pt
        rfm_errs.append(fla.chordal_dist(real_flag_mean, center_pt, flag_type))

        rfmed_errs.append(fla.chordal_dist(real_flag_median, center_pt, flag_type))

    # plot results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 3))
    ax1.plot(noises, rfmed_errs, marker = 's', label = 'Flag Median (Ours)')
    ax1.plot(noises, rfm_errs, marker = 'x', label = 'Flag Mean (Ours)')
    ax1.set(xlabel='Outlier Ratio', ylabel='Chordal Distance from Center')
    ax1.legend()

    outlier_results = pd.DataFrame()
    outlier_results['Outlier Ratio'] = noises
    outlier_results['FL-Mean (Ours)'] = rfm_errs
    outlier_results['FL-Median (Ours)'] = rfmed_errs


    # plt.savefig('synth_flags_outliers.pdf')

    m_errs = []
    med_errs = []
    rfm_errs = []
    rfmed_errs = []
    n_errs = []
    e_errs = []

    noises = []


    for exp in range(1,45,5):
        noise = exp/50
        noises.append(noise)

        data, center_pt = generate_flag_data_noisy(n_pts, n, flag_type, noise)
        stacked_data = np.stack(data, axis = 2)

        real_flag_mean = fla.flag_mean(stacked_data,  flag_type = flag_type)      

        real_flag_median = fla.flag_median(stacked_data,  flag_type = flag_type, max_iters = 100)

        #distances to center pt
        rfm_errs.append(fla.chordal_dist(real_flag_mean, center_pt, flag_type))

        rfmed_errs.append(fla.chordal_dist(real_flag_median, center_pt, flag_type))


    # plot results
    ax2.plot(noises, rfmed_errs, marker = 's', label = 'FL-Median (Ours)')
    ax2.plot(noises, rfm_errs, marker = 'x', label = 'FL-Mean (Ours)')

    ax2.set(xlabel='Noise')

    

    plt.tight_layout()
    plt.savefig('flagmeancompare.pdf')

    noise_results = pd.DataFrame()
    noise_results['Noise'] = noises
    noise_results['FL-Mean (Ours)'] = rfm_errs
    noise_results['FL-Median (Ours)'] = rfmed_errs
